In [ ]:
#Import ee and required packages
import ee
ee.Initialize()
import pandas as pd 
import numpy as np
import glob
import geopandas as gpd
import time

from src.gee_funs import *
import src.build_annual_cube as bac
import src.preprocess_points as pp

In [ ]:
# VARIABLE DECLARATIONS

STATE="Montana"
state_abbrevs = {
    'Montana' : 'MT'
}

# years to export for both the Yearly Covariate Images
# and the Yearly Training CSVS
# Note we are limited to 2002 - 2018 due to the water year covariate  
start_year = 2002
end_year = 2018
years = range(start_year, end_year)
years_strings = [str(y) for y in years]

gee_path='users/kjchristensen93/'
ais_point_path = gee_path + 'EBT_data/EBT_mfish_data_presence_heuristic'
ais_thinned_point_path = ais_point_path + '_sthin'

#Define export locations:
#GEE yearly covariate folder
assetId = (gee_path+'covariates/covariates') 
#User training csv local directory folder
trainingdata = ('./datasets/training/')
trainingglob = trainingdata+'*.csv'
#decadal CSV directory and naming conventions
decade1 = ('./datasets/decade/decade1.csv')
decade2 =('./datasets/decade/decade2.csv')

In [ ]:
#If you need to create the spatially thinned asset...Otherwise skip to Define Modular Variables below
#Define GEE asset/location of desired dataset 
#(Formatted CSV must be uploaded to your GEE assets with Lat/Long columns defined before starting)
if False:
    pp.spatially_thin(ais_point_path, start_year, end_year)

In [ ]:
# If you have a spatially thinned data set, start here after initializing ee
# embed observation Year as system:start_time for thinned dataset 
# We have had to add this "Year Column" manually to the datasets.  
# Make sure your dataset has correct column headings
SThin = ee.FeatureCollection(ais_thinned_point_path).map(embed_date)

In [ ]:
# Define geometry by changing state name so we can export the whole state at once
state_geometry = ee.FeatureCollection("TIGER/2016/States").filter(ee.Filter.eq('NAME',STATE)).geometry()

# Choose state to clip HUC by. Change Abbreviation to match dataset 
#Enter state full name for X (i.e., Illinois/ look at dataset for formats for this stuff)
HUC_clip = ee.FeatureCollection("USGS/WBD/2017/HUC12").filter(ee.Filter.eq('states',state_abbrevs[STATE]))

In [ ]:
#Skip this step if you already have them stored in GEE
#Export Yearly Covariate Images

#========================================================
# Run covariate algorithm and build a list of images
# with each image corresponding to each year and each band corresponding to each covariate
#========================================================
banded_images_list = bac.build_all_cubes(start_year, end_year)

# Export each image within the for loop
for i,y in zip(range(len(years_strings)), years_strings):
    print("Starting", y)
    img = ee.Image(banded_images_list.get(ee.Number(i)))
    export = ee.batch.Export.image.toAsset(
        image = img,
        description = 'covariates'+y,
        assetId = assetId + y, 
        region = ee.Geometry(state_geometry),
        scale =  100,
        maxPixels = 1e13)
    export.start()
    
    print(y,"status:    ", export.status()['state'])

    # Wait for 30 seconds so that the export['state'] gives insightful information
    time.sleep(15)
    # TODO better handle status messages
    # while (export.status()['state'] == 'READY'):
    #    time.sleep(15)
        
    print(y,"status:    ", export.status()['state'])
    
    # If this status is "RUNNING", then there are no egretious syntax errors. 
    # However, it is still possible that these export commands fail after more than 30 seconds.
    # In that case, it is likely that there is a Computation Time Out Error (remember exporting the annual stacks)
    time.sleep(15)
    print(y,"status:    ", export.status()['state'])
    

In [ ]:
#Start Here if you have yearly covariates created

#Export training CSVs
## Reduce Regions from existing images

# COVARIATE IMAGES  
images = list(map(lambda x: ee.Image(assetId + str(x)), years))

for i in range(len(years)):
    print("Starting", start_year+i)
    
    data = reduce_HUCS(images[i],SThin,HUC_clip) 
    
    ## PYTHON API MAGIC!! LOOK HERE
    my_csv = pd.DataFrame([x['properties'] for x in data.getInfo()['features']])
    
    # From there, we can write it directly to our directory and stitch it together afterwards
    my_csv.to_csv((trainingdata) + str(2002+i) + '.csv', index=False) 
    print("Finished", start_year+i)

In [ ]:
#export the information that we will use to project habitat suitability. 
#Decades were convenient for RBT, but not other taxa with less data/ we can change.
# Change to match dataset
# maybe we should think about 2 and 5 year bins due to limitations of datasets for some 
# taxa / to make more useful for managers

write_predictions_input_csv(images[:7], decade1, HUC_clip)  

# write_predictions_input_csv(images[7:], decade2, HUC_clip)

In [ ]:
import src.ml_funs